In [2]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, power_transform, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.ensemble import RUSBoostClassifier, BalancedRandomForestClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data

In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [4]:
models = {}
for im,samp in IMBS.items():
    for c,clf in CLFS.items():
        models[(im,c)] = Pipeline([('samp',samp),('clf',clf)])
    
models.keys(),len(models)

(dict_keys([('smote', 'dt'), ('smote', 'lr'), ('smote', 'nb'), ('smote', 'svm'), ('smote', 'knn'), ('smote', 'rf'), ('rus', 'dt'), ('rus', 'lr'), ('rus', 'nb'), ('rus', 'svm'), ('rus', 'knn'), ('rus', 'rf'), ('wilson', 'dt'), ('wilson', 'lr'), ('wilson', 'nb'), ('wilson', 'svm'), ('wilson', 'knn'), ('wilson', 'rf'), ('tomek', 'dt'), ('tomek', 'lr'), ('tomek', 'nb'), ('tomek', 'svm'), ('tomek', 'knn'), ('tomek', 'rf'), ('None', 'dt'), ('None', 'lr'), ('None', 'nb'), ('None', 'svm'), ('None', 'knn'), ('None', 'rf')]),
 30)

In [5]:
path = "Noisy.csv"
cols = pd.MultiIndex.from_product([IMBS.keys(),CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
#df = pd.read_csv(path,header=[0,1,2],index_col=0)

In [ ]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv(path)

0
activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
('smote', 'dt')
3.44 [0.155, 0.237]
('smote', 'lr')
13.32 [0.258, 0.298]
('smote', 'nb')
0.35 [0.289, 0.394]
('smote', 'svm')
180.96 [0.245, 0.194]
('smote', 'knn')
3.36 [0.205, 0.265]
('smote', 'rf')
18.27 [0.235, 0.281]
('rus', 'dt')
0.24 [0.183, 0.411]
('rus', 'lr')
2.13 [0.252, 0.294]
('rus', 'nb')
0.19 [0.283, 0.39]
('rus', 'svm')
1.48 [0.262, 0.251]
('rus', 'knn')
0.67 [0.207, 0.265]
('rus', 'rf')
2.83 [0.252, 0.278]
('wilson', 'dt')
8.94 [0.225, 0.311]
('wilson', 'lr')
9.67 [0.282, 0.316]
('wilson', 'nb')
6.44 [0.283, 0.415]
('wilson', 'svm')
19.7 [0.278, 0.315]
('wilson', 'knn')
8.01 [0.224, 0.241]
('wilson', 'rf')
10.86 [0.285, 0.33]
('tomek', 'dt')
6.43 [0.199, 0.263]
('tomek', 'lr')
9.81 [0.269, 0.327]
('tomek', 'nb')
5.55 [0.293, 0.391]
('tomek', 'svm')
25.24 [0.27, 0.288]
('tomek', 'knn')
7.32 [0.225, 0.26]
('tomek', 'rf')
10.79 [0.281, 0.337]
('None', 'dt')
1.13 [0.16, 0.228]
('None', 'lr')
3.77 [0.25

In [ ]:
df.shape

## Effect of Noise on Performance

In [ ]:
noise = pd.read_csv("Imb X Clf- Noise.csv",header=[0,1,2],index_col=0)
clean = pd.read_csv("Imb X Clf- Clean.csv",header=[0,1,2],index_col=0)
noise.shape,clean.shape

In [ ]:
wilcoxon(noise.values.reshape(-1),clean.values.reshape(-1))

In [ ]:
noise.values.mean(),clean.values.mean()

## Statistical Analysis

In [ ]:
from scipy.stats import wilcoxon, friedmanchisquare
import scikit_posthocs as sp

In [ ]:
boo = df.rename_axis('Datasets').reset_index()

In [ ]:
res = pd.melt(boo,id_vars=['Datasets'])
res.columns

In [ ]:
res

### Compare DT & RF & NB

In [ ]:
mathew =  res[res['metric']=='matthews_corrcoef'].drop(columns=['metric'])

In [ ]:
mathew = mathew[mathew['clf'].isin(['dt','rf','nb'])]
mathew.shape

In [ ]:
tmp = mathew.set_index(['Datasets','imb'])
tmp

In [ ]:
hey = tmp.pivot(columns='clf')
hey

In [ ]:
dog = hey.reset_index(drop=True)
dog.columns = dog.columns.droplevel(0)
dog

In [ ]:
friedmanchisquare(*(dog[c] for c in dog.columns))

In [ ]:
dog.mean()f

In [ ]:
pvals = pd.DataFrame(columns=dog.columns,index=dog.columns,dtype='float')
for c in dog.columns:
    for d in dog.columns:
        if c==d: continue
        pvals.loc[c,d] = float(wilcoxon(dog[c],dog[d]).pvalue)
pvals

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
ps = pvals.values.reshape(-1)
idx = np.isnan(ps)
idx

In [ ]:
ps[~idx]

In [ ]:
multipletests(ps[~idx],method='fdr_by')